# ML Models

## Import Libraries & Load Dataframe from AWS DB

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from math import sin, cos, sqrt, atan2, radians

import query_helper
import get_new_route
import json


In [32]:
df_numeric = pd.read_csv('df.csv', index_col='id')
#reorder columns
df_numeric =df_numeric[['name', 'rating', 'stars', 'starVotes', 'pitches', 'location', 'region',
                               'area', 'sub_area', 'wall', 'longitude', 'latitude', 'url', 'Sport',
                               'Trad', 'Boulder', 'TR', 'Alpine', 'Aid', 'Ice', 'Snow', 'Mixed',
                               'danger', 'rope_grade', 'boulder_grade', 'infos', 'slab', 'traverse',
                               'roof', 'corner', 'crack', 'face', 'flake', 'fingers', 'jug', 'exposed',
                               'dihedral', 'sustained', 'technical', 'run out', 'well protected',
                               'chimney', 'offwidth', 'stem', 'arete', 'crimp', 'vertical', 'powerful',
                               'in_range']]
df_numeric.head()

,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,...,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
id,,,,,,,,,,,,,,,,,,,,,
105714722,Central Yellow Wall,V3 R,4.4,22,0,South Dakota,Custer State Park,Sylvan Lake,Sylvan Lake Bouldering,Campground Boulder,...,0,0,0,0,0,0,0,0,0,1
105714728,Waves,5.8,4.6,217,2,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Shipyard Rock,...,0,0,0,0,0,0,0,1,0,1
105714731,East Chimney Variation,5.7,4.1,40,2,South Dakota,Custer State Park,Cathedral Spires,Station 13,0,...,0,0,0,0,0,0,0,0,0,1
105714734,Conn Diagonal,5.7,4.9,151,3,South Dakota,Custer State Park,Sylvan Lake,Outlets,Outer Outlet,...,0,0,0,0,1,0,0,0,0,1
105714737,Bolts for Bobs,5.8,3.5,125,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Borneo,...,0,0,0,0,0,0,0,0,0,1


## Get input from user for recommendation

In [33]:
target_id = 106875741
# target_lat = 32.9127 
# target_lon = -116.882
target_state =''
target_city =''
target_zipcode = '92008'
target_radius_range=60
star_limit = 3.5
###other parameters to be added here later

In [34]:
type('ss') == str

True

### Get coordinates for zip or city

In [35]:
with open('us-zip-code-latitude-and-longitude.json') as f:
  coord_dict = json.load(f)

In [36]:
def get_coords(target_city=None, target_state=None, zipcode=None):
    #find the coordinates for city or zip code
    for city in coord_dict:
        if city['fields']['zip']==zipcode:
            return city['fields']['latitude'],city['fields']['longitude']
        if (city['fields']['state']==target_state)&(city['fields']['city']==target_city):
            return city['fields']['latitude'],city['fields']['longitude']
    #if nothing is found return none
    return None, None

In [37]:
target_lat, target_lon = get_coords(target_city, target_state, target_zipcode)

In [38]:
print(target_lat, target_lon)

33.158137 -117.32646


### Create fxn to see if climb is in search range

In [39]:
#function takes search param range and assigns to original df if climb in_range
def in_range(df_fxn, lat, lon, radius_range=None):
    if radius_range:
        R= 3958.8 
        if (lat == None)|(lon==None):
            df_fxn['in_range'] = 1
        else:
            #assign target coords and set to radians for calc
            lat1 = radians(lat)
            lon1 = radians(lon)
            for index, row in df_fxn.iterrows():
                #assign the lat and lon for each climb
                lat2 = radians(row['latitude'])
                lon2 = radians(row['longitude'])

                dlon = lon2 - lon1
                dlat = lat2 - lat1

                a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
                c = 2 * atan2(sqrt(a), sqrt(1 - a))

                distance = R * c

                #assign in_range col to 1 if the climb is in range
                if distance < radius_range:
                    df_fxn.at[index,'in_range']=1
                else:
                    df_fxn.at[index,'in_range']=0   
    else:
        df_fxn['in_range'] =1



In [40]:
def star_cutoff(df_fxn, star_limit=3.5):
    for index, row in df_fxn.iterrows():
        #assign in_range col to 1 if the climb is in range
        if (df_fxn.at[index, 'stars'] >= star_limit)&(df_fxn.at[index, 'in_range']!=0):
            df_fxn.at[index,'in_range']=1
        else:
            df_fxn.at[index,'in_range']=0   

In [41]:
df_numeric.iloc[0,:]

name                                            Central Yellow Wall
rating                                                         V3 R
stars                                                           4.4
starVotes                                                        22
pitches                                                           0
location                                               South Dakota
region                                            Custer State Park
area                                                    Sylvan Lake
sub_area                                     Sylvan Lake Bouldering
wall                                             Campground Boulder
longitude                                                  -103.564
latitude                                                    43.8463
url               https://www.mountainproject.com/route/10571472...
Sport                                                             0
Trad                                            

In [42]:
df_numeric.loc[target_id,:]

name                                                    Via Leni 6b
rating                                                        5.10b
stars                                                             5
starVotes                                                         4
pitches                                                           6
location                                              International
region                                                       Europe
area                                                    Switzerland
sub_area                                             Albigna Valley
wall                                                 Spazzacaldeira
longitude                                                    9.6383
latitude                                                    46.3387
url               https://www.mountainproject.com/route/10687574...
Sport                                                             1
Trad                                            

In [43]:
# ##difficulity cutoff, function not run until climb is in df
# def diff_cutoff(df_fxn, delta=6, target_grade):
#     if df_fxn.loc[target_id,'Boulder']==0:
#         target_grade = df_fxn.loc[target_id,'rope_grade']
#         for index, row in df_fxn.iterrows():
#             #assign in_range col to 1 if the climb is in range
#             if (df_fxn.at[index, 'rope_grade'] <= target_grade+delta)&(df_fxn.at[index, 'rope_grade'] >= target_grade-delta)
#                 &(df_fxn.at[index, 'in_range']!=0):
#                 df_fxn.at[index,'in_range']=1
#             else:
#                 df_fxn.at[index,'in_range']=0                

### Call function to assign if climb in range

In [44]:
## used to get list of climbs allowed for comparison
in_range(df_numeric, lat = target_lat, lon = target_lon, radius_range=target_radius_range)

### Star cutoff (ie only give results for routes with above 3.5 stars)

In [45]:
star_cutoff(df_numeric, star_limit)

In [46]:
df_numeric.in_range.value_counts()

0    35242
1     1372
Name: in_range, dtype: int64

### To begin, see if if the climb already exists in db

In [47]:
if target_id in df_numeric.index:
    print('We have climb already')
    #make sure reference climb is assigned in_range
    df_numeric.loc[target_id,'in_range']=1
else:
    print('Making API call and Scraping climb data')
    if(get_new_route.get_route_details(target_id)):
        #the function in the if statement saves target climb to target_climb.csv and returns 1
        df_target= pd.read_csv('target_climb.csv', index_col= 'id')
        df_target.drop(columns=['Unnamed: 0'], inplace=True)
        df_target['in_range'] = 1
        #order the same as df_numeric columns
        df_target = df_target[['name', 'rating', 'stars', 'starVotes', 'pitches', 'location', 'region',
                               'area', 'sub_area', 'wall', 'longitude', 'latitude', 'url', 'Sport',
                               'Trad', 'Boulder', 'TR', 'Alpine', 'Aid', 'Ice', 'Snow', 'Mixed',
                               'danger', 'rope_grade', 'boulder_grade', 'infos', 'slab', 'traverse',
                               'roof', 'corner', 'crack', 'face', 'flake', 'fingers', 'jug', 'exposed',
                               'dihedral', 'sustained', 'technical', 'run out', 'well protected',
                               'chimney', 'offwidth', 'stem', 'arete', 'crimp', 'vertical', 'powerful',
                               'in_range']]
        
        df_numeric = pd.concat([df_numeric, df_target])
    else:
        print("Something went wrong")


We have climb already


In [48]:
df_numeric.loc[target_id,:]

name                                                    Via Leni 6b
rating                                                        5.10b
stars                                                             5
starVotes                                                         4
pitches                                                           6
location                                              International
region                                                       Europe
area                                                    Switzerland
sub_area                                             Albigna Valley
wall                                                 Spazzacaldeira
longitude                                                    9.6383
latitude                                                    46.3387
url               https://www.mountainproject.com/route/10687574...
Sport                                                             1
Trad                                            

In [49]:
df_numeric.tail()

,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,...,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
id,,,,,,,,,,,,,,,,,,,,,
118210398,Dead and Bloated,V5-6,5.0,1,0,California,Southern Sierra,San Joaquin River Gorge,Downstream,Dead and Bloated Boulder,...,0,0,0,0,0,0,0,0,0,0
118210488,Unknown #1,V0-1,3.0,1,0,California,Lake Tahoe,I-80 Corridor,Auburn and Grass Valley,"Auburn SRA,Tall Green Bridge,Riverside Boulder...",...,0,0,0,0,0,0,0,0,0,0
118210496,Unknown #2,V1-2,3.0,1,0,California,Lake Tahoe,I-80 Corridor,Auburn and Grass Valley,"Auburn SRA,Tall Green Bridge,Riverside Boulder...",...,0,0,0,0,0,0,0,0,0,0
118210504,Unknown #3,V1-2,3.0,1,0,California,Lake Tahoe,I-80 Corridor,Auburn and Grass Valley,"Auburn SRA,Tall Green Bridge,Riverside Boulder...",...,0,0,0,0,0,1,0,0,0,0
118211517,Mertensia Pillar,WI5,4.0,1,1,Colorado,CO Ice & Mixed,RMNP - Mixed/Ice,Wild Basin,Mertensia Falls,...,0,0,0,0,0,0,0,0,0,0


### Diff_grade cutoff WIP

In [50]:
###WIP call grade range function
# pass in deets for target climb

## Reccomender

#### Kernel Imports

In [51]:
# Import kernels
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import laplacian_kernel
from sklearn.metrics.pairwise import chi2_kernel



In [52]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

#### Create df_in_range to run recommender in subset

In [53]:
df_in_range = df_numeric[df_numeric['in_range']==1].reset_index()      
target_index =df_in_range.index[df_in_range['id']==target_id][0] #store target climb index in subset that will be compared
df_in_range.shape

(1373, 50)

In [56]:
target_index

493

In [57]:
df_in_range.tail()

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,...,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
1368,118175687,Aerial Chariot Direct,V7,5.0,1,0,California,San Diego and Environs,North San Diego County,Lake Ramona,...,0,0,0,0,0,0,0,0,0,1
1369,118189308,Golgotha,V8-9,5.0,1,0,California,San Diego and Environs,North San Diego County,Lake Ramona,...,0,0,0,0,0,1,1,0,0,1
1370,118192124,Bloody L,5.7,4.0,2,1,California,Inland Empire,Big Rock Area,L-Slab/Lakeview Slab,...,0,0,0,0,0,0,0,0,0,1
1371,118202461,Pieces of Eight,5.10c/d PG13,4.0,2,1,California,Tahquitz & Suicide Rocks,Suicide Rock,(l) Buttress of Cracks,...,0,0,0,0,0,0,0,0,0,1
1372,118208812,Coyote Traverse,5.8 V0+ PG13,3.5,2,1,California,San Diego and Environs,North San Diego County,Calavera lake,...,0,0,0,0,0,0,0,0,0,1


### Scale Features

#### Create Features DF

In [58]:
#creates features from df_in_range used for comparison
features = df_in_range.loc[:,['stars', 'pitches', 'Sport', 'Trad', 'Boulder', 'TR', 'Alpine', 'Aid',
       'Ice', 'Snow', 'Mixed', 'danger', 'rope_grade', 'boulder_grade', 'slab', 'traverse', 'roof', 
                'corner', 'crack', 'face','flake', 'fingers',
                 'jug', 'exposed', 'dihedral', 'sustained', 'technical','run out', 'well protected',
                 'chimney', 'offwidth', 'stem', 'arete','crimp', 'vertical', 'powerful']] #,'longitude','latitude',

In [59]:
features.shape

(1373, 36)

In [60]:
# features.iloc[:,[12,13]]

#### Pick scaling type (AND UPDATE WEIGHTS)

In [61]:
min_max_scaler = MinMaxScaler()
scalar = StandardScaler()

In [62]:
##### Pick a scaling option ###############################

# features_scaled = scalar.fit_transform(features)
# features_scaled = min_max_scaler.fit_transform(features.drop(columns=['danger','pitches']))

features_scaled = min_max_scaler.fit_transform(features)

# scale danger and pitches using ss and add into features scaled df
# features_scaled = np.concatenate((features_scaled, scalar.fit_transform(features[['danger', 'pitches']])), axis=1)

##################################################################

for i in range(features_scaled.shape[0]):
    features_scaled[i][10]=features_scaled[i][12]*10  #weight rope_grade higher
    features_scaled[i][11]=features_scaled[i][13]*10 #weight boulder_grade higher

### Now lets fit the similarity model

#### Rec function

In [63]:
def get_recommendations(idx, kernel_type):

    #value to store scores and indicies
    score_matrix = np.ndarray(shape=(len(df_in_range),2), dtype=float)

    #go through the target climb vs all onthers in our db and populate score mtx with index and similarity
    for i in range(df_in_range.shape[0]):
        score = kernel_type(features_scaled[idx].reshape(1,-1),features_scaled[i].reshape(1,-1))
        score_matrix[i][0] =  i        ##the index comparison corresponding to the score
        score_matrix[i][1] = score     ##the score for the current index

    # Sort the climbs based on the similarity scores
    score_matrix = sorted(score_matrix, key=lambda x: x[1], reverse=True)
    

#########################  WIP ADD/calculate SIMilarity VALUE from tf-idf infos comparison #######################

    # # Get the scores of the 20 most similar climbs
    score_matrix = score_matrix[1:11]

    # # Get the climb indices (& cast to ints)
    climb_indices = [int(i[0]) for i in score_matrix]
    
    # Return the top 20 most similar climbs
    return df_in_range.loc[climb_indices,:]

#### Cells for comparison (delete later)

In [64]:
df_numeric.loc[target_id, :]

name                                                    Via Leni 6b
rating                                                        5.10b
stars                                                             5
starVotes                                                         4
pitches                                                           6
location                                              International
region                                                       Europe
area                                                    Switzerland
sub_area                                             Albigna Valley
wall                                                 Spazzacaldeira
longitude                                                    9.6383
latitude                                                    46.3387
url               https://www.mountainproject.com/route/10687574...
Sport                                                             1
Trad                                            

In [65]:
target_index

493

#### Call rec fxn

In [75]:
pd.set_option('max_colwidth', 100)

In [76]:
rec=get_recommendations(target_index, cosine_similarity)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,infos,slab,traverse,roof,corner,crack,face,flake,fingers,jug,exposed,dihedral,sustained,technical,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
208,105913864,Dont be Fooled,5.10d,5.0,1,1,California,San Diego and Environs,South San Diego County,Otay Mesa,0,-116.907,32.6030,https://www.mountainproject.com/route/105913864/dont-be-fooled,1,0,0,0,0,0,0,0,0,0,28,0,The last route to looker's left on the south side of the canyon. Bouldery but short-lived.-5 bolts,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
167,105835122,Triton Tower,5.11a,4.5,68,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Triton Tower",-116.818,32.9043,https://www.mountainproject.com/route/105835122/triton-tower,1,0,0,0,0,0,0,0,0,0,29,0,Climb that climbs the obvious tower of the left wall. The first pitch is 5.8 and goes to the bas...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
984,111177054,Tears of Envy,5.11b,4.9,26,1,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Center/Headwall",-116.818,32.9046,https://www.mountainproject.com/route/111177054/tears-of-envy,1,0,0,0,0,0,0,0,0,0,32,0,"Tears of Envy is a brilliant line that ascends the black, green and orange headwall of ECM. This...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
366,106306476,Godzilla,5.10b,4.2,11,1,California,San Diego and Environs,North San Diego County,Poway Crags,"Poway Crags Proper,Godzilla Buttress",-116.949,32.9850,https://www.mountainproject.com/route/106306476/godzilla,1,0,0,0,0,0,0,0,0,0,22,0,An awesome route. Solid moves and epic climb that is a must do at the Poway crag. You can run it...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
280,106100920,Blue Collar,5.10c,4.3,6,1,California,Inland Empire,Box Springs Mountain Park,Knob Wall,0,-117.309,33.9889,https://www.mountainproject.com/route/106100920/blue-collar,1,0,0,0,0,0,0,0,0,0,25,0,This is one of the best routes in Box Springs! Starting just below the right end of the horizon...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
851,108807165,Thunder Bunny,5.11a,4.5,2,1,California,San Diego and Environs,North San Diego County,Old Castle,Main Wall (aka The Gorilla Head Buttress),-117.124,33.2539,https://www.mountainproject.com/route/108807165/thunder-bunny,1,0,0,0,0,0,0,0,0,0,29,0,"One of the better sport climbs in north county, Thunder Bunny climbs continuous bulges via techn...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1253,116180423,Randall Handle,5.9,4.0,1,1,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Not So Grassy Ledge",-116.818,32.9045,https://www.mountainproject.com/route/116180423/randall-handle,1,0,0,0,0,0,0,0,0,0,17,0,Consistent climbing on high quality stone at a moderate grade make for fun climbing past the nam...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
407,106545739,Swirly Bird,5.12a,5.0,1,1,California,Inland Empire,Box Springs Mountain Park,Sleepers Wall,0,-117.309,33.9892,https://www.mountainproject.com/route/106545739/swirly-bird,1,0,0,0,0,0,0,0,0,0,39,0,"Head up the brown slab, then pull the small bulge directly in the center of the wall. Hard pinc...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1008,111571633,Emerald City,5.10a/b,4.0,30,1,California,Inland Empire,Riverside Quarry,(h) Rubble Row,0,-117.417,34.0169,https://www.mountainproject.com/route/111571633/emerald-city,1,0,0,0,0,0,0,0,0,0,21,0,This line follows the green bolts leading up the obvious crack-like feature on the overhanging h...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1273,116357491,Rockaledge,5.10b,4.0,1,1,California,San Diego and Environs,North San Diego County,Mt. Gower,"Middle Area,Grit Wall",-116.781,33.0114,https://www.mountainproject.com/route/116357491/rockaledge,1,0,0,0,0,0,0,0,0,0,22,0,Rat

In [68]:
rec=get_recommendations(target_index, rbf_kernel)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,...,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
366,106306476,Godzilla,5.10b,4.2,11,1,California,San Diego and Environs,North San Diego County,Poway Crags,...,0,0,0,0,0,0,0,0,0,1
1273,116357491,Rockaledge,5.10b,4.0,1,1,California,San Diego and Environs,North San Diego County,Mt. Gower,...,0,0,0,0,0,0,0,0,0,1
1008,111571633,Emerald City,5.10a/b,4.0,30,1,California,Inland Empire,Riverside Quarry,(h) Rubble Row,...,0,0,0,0,0,0,0,0,0,1
280,106100920,Blue Collar,5.10c,4.3,6,1,California,Inland Empire,Box Springs Mountain Park,Knob Wall,...,0,0,0,0,0,0,0,0,0,1
364,106306456,Don't Pet The Tiger,5.10b,3.7,11,1,California,San Diego and Environs,North San Diego County,Poway Crags,...,0,0,0,0,0,0,0,0,0,1
490,106866127,Khazaddum,5.10b,3.7,67,1,California,San Diego and Environs,South San Diego County,Mission Gorge,...,0,0,0,0,0,0,0,0,0,1
313,106152433,Snapping Turtle,5.10b,3.6,5,1,California,San Diego and Environs,North San Diego County,Poway Crags,...,0,0,0,0,0,0,0,0,0,1
517,106986499,Wish You Were Here,5.10a,3.8,12,1,California,San Diego and Environs,South San Diego County,Corte Madera,...,0,0,0,0,0,0,0,0,0,1
208,105913864,Dont be Fooled,5.10d,5.0,1,1,California,San Diego and Environs,South San Diego County,Otay Mesa,...,0,0,0,0,0,0,0,0,0,1
1253,116180423,Randall Handle,5.9,4.0,1,1,California,San Diego and Environs,South San Diego County,El Cajon Mountain,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
rec=get_recommendations(target_index, laplacian_kernel)
rec

In [ ]:
# rec=get_recommendations(target_index, euclidean_distances)
# rec

In [ ]:
# rec=get_recommendations(target_index, linear_kernel)
# rec

In [ ]:
# rec=get_recommendations(target_index, polynomial_kernel)
# rec

In [ ]:
# rec=get_recommendations(target_index, sigmoid_kernel)
# rec

In [ ]:
# rec=get_recommendations(climb_id, chi2_kernel)
# rec

## NLP (work in progress)

### Import NLP Data (redundent delete later)

In [ ]:
df_nlp = query_helper.query_to_df('SELECT * FROM route_description;')
df_nlp.set_index('id', inplace=True)
df_nlp.head()

In [ ]:
key_words = ['slab', 'traverse', 'roof', 'corner', 'ledge', 'crack', 'face','flake', 'finger', 'fingers',
             'hand', 'hands', 'arch', 'balancy', 'balance', 'jug', 'squeeze', 'mantel', 'sustained',  
             'ramp', 'overhung', 'dihedral', 'sporty', 'heady', 'pump', 'pumpy', 'technical',
             'run out', 'mental', 'well protected', 'chimney', 'offwidth', 'stem', 'arete', 'exposed', 'exposure',
             'crimp','crimpy', 'vertical', 'slabby', 'cave', 'steep', 'bouldery'. 'powerful']

In [ ]:
col_key_words = ['slab', 'traverse', 'roof', 'corner', 'ledge', 'crack', 'face','flake', 'finger',
             'hand', 'arch', 'balancy', 'jug', 'squeeze', 'mantel', 'exposed', 
             'ramp', 'overhung', 'dihedral', 'sporty', 'sustained','pump', 'technical',
             'run out', 'mental', 'well protected', 'chimney', 'offwidth', 'stem', 'arete',
             'crimp', 'vertical', 'cave', 'steep', 'bouldery', 'powerful']

In [ ]:
df_numeric

In [ ]:
df_full = df_numeric.join(df_nlp)
df_full.head()

In [ ]:
df_numeric.head()

### Break descriptions into rope and boulder

#### Rope

In [ ]:
df_sub = df_numeric[(df_numeric['Boulder']==0)]
df_sub = df_sub[(df_numeric['Ice']==0)]
df_sub = df_sub[(df_numeric['Snow']==0)]

df_sub.head()

In [ ]:
df_sub.Boulder.value_counts()

In [ ]:
len(df_sub.index)

In [ ]:
len(df_nlp.loc[df_sub.index])

In [ ]:
nlp_rope =df_nlp.loc[df_sub.index]

In [ ]:
nlp_rope.head()

#### Boulder

#### Tokenizer

In [ ]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

In [ ]:
extra_stop = ['climb','climbing', 'crux', 'rope', 'leave', 'use', 'start', 'end',
              'look', 'rock', 'tree', 'follow', 'continue', 'belay', 'photo', 'add', 'climber', 'route', 
              'lot', 'anchor', '...', '1', '2', '3', '4', '--', 'pitch', 'page', 'cold', 'hot', 'warm',
              'belayer', 'fun', 'like', 'unknown', 'rap', 'left', 'right', 'wide', 'leader']


In [ ]:
for word in extra_stop:
    stop_words.add(word)

In [ ]:
stop_words

In [ ]:
# Creating our tokenizer function
def spacy_tokenizer(text):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(text)
    
#     mytokens = [word for word in mytokens if word.pos_ != "PROPN"]
    
    mytokens = [ word if word.pos_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

### TF-IDF Vectorizer

#### Run the vectorizer

In [ ]:
df_nlp.loc[105714722,:]

In [ ]:
df_nlp[df_nlp.isna().values]

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(tokenizer=spacy_tokenizer, min_df=5, max_df=.7)


#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df_nlp['infos'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

#### Save it to load later

In [ ]:
# Save the trained model as a pickle string. 
from sklearn.externals import joblib 
import pickle
tfidf_5_7 = pickle.dumps(tfidf_matrix) 

  
# Save the model as a pickle in a file 
joblib.dump(tfidf_matrix, 'tfidf_5_7.pkl') 
  
# Load the model from the file 
# tfidf_loaded = joblib.load('tfidf_5_7.pkl')  
  

### Get original index

In [ ]:
#get index to iterate over
search_range = df_in_range.orig_index

# get index for target in reference to whole matrix
nlp_target_index = int(df_in_range.orig_index[df_in_range['id']==target_id].values)
print('target_index:',nlp_target_index)

In [ ]:
df_nlp.loc[105793305]

In [ ]:
######NEED TO COMBINE WITH get_recommendations so we can get average weighted score#########################
def get_recommendations_for_nlp(idx, kernel_type):

    #value to store scores and indicies
    score_matrix = np.ndarray(shape=(len(df_in_range),2), dtype=float)
    
    for iter_,i in enumerate(search_range.values):
        score = kernel_type(tfidf_matrix[idx],tfidf_matrix[i])
        score_matrix[iter_][0] =  i        ##the index comparison corresponding to the score
        score_matrix[iter_][1] = score     ##the score for the current index

    # Sort the climbs based on the similarity scores
    score_matrix = sorted(score_matrix, key=lambda x: x[1], reverse=True)
    
    # score_matrix
#########################  ADD/calculate SIMilarity VALUE   ###########################

    # # Get the scores of the 20 most similar climbs
    score_matrix = score_matrix[1:20]

    # # Get the climb indices (& cast to ints)
    climb_indices = [int(i[0]) for i in score_matrix]
    
    # Return the top 20 most similar climbs
    return df_numeric.iloc[climb_indices,:]

In [ ]:
rec = get_recommendations_for_nlp(nlp_target_index, cosine_similarity)
rec

In [ ]:
df_nlp.loc[rec.index]

### LDA

#### CV

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(tokenizer=spacy_tokenizer, max_df=0.90, min_df=5, stop_words='english', ngram_range=(1,1))

In [ ]:
dtm = cv.fit_transform(nlp_rope['info'].sample(n=1000, random_state=1))

#### LDA Model

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=32,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

print(lda_model)  # Model attributes

In [ ]:
# This can take awhile, we're dealing with a large amount of documents!

lda_output = lda_model.fit_transform(dtm)


#### Diagnose model performance with perplexity and log-likelihood
A model with higher log-likelihood and lower perplexity (exp(-1. * log-likelihood per word)) is considered to be good. Let’s check for our model.



In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

In [ ]:
len(lda_model.components_)

In [ ]:
single_topic = lda_model.components_[0]

In [ ]:
top_word_indices = single_topic.argsort()[-10:]

In [ ]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

#### Top words for all groups

In [ ]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-30:]])
    print('\n')

In [ ]:
extra_stop = ['climb','climbing', 'crux', 'rope', 'leave', 'use', 'start', 'end',
              'look', 'rock', 'tree', 'follow', 'continue', 'belay', 'photo', 'add', 'climber', 'route', 
              'lot', 'anchor', '...', '1', '2', '3', '4', '--', 'pitch', 'page', 'cold', 'hot', 'warm',
              'belayer', 'fun', 'like', 'unknown', 'rap', 'left', 'right']



### Try new reverse tokenizer

In [ ]:
key_words = ['slab', 'traverse', 'roof', 'corner', 'ledge', 'crack', 'face','flake', 'bolt', 'finger', 'fingers'
             'cam', 'camalot', 'hand', 'hands', 'arch', 'balancy', 'jug', 'squeeze', 'mantel', 'sustained', 'nut', 
             'gear', 'ramp', 'overhung', 'balance', 'dihedral', 'sporty', 'heady', 'pump', 'pumpy', 'technical',
             'run out', 'mental', 'well protected', 'chimney', 'offwidth', 'stem', 'arete', 'exposed', 'exposure',
             'crimp','crimpy', 'vertical', 'slabby', 'cave', 'steep', 'sidepull','bouldery']

In [ ]:
# Creating our tokenizer function
def spacy_tokenizer_reverse(text):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(text)
    
#     mytokens = [word for word in mytokens if word.pos_ != "PROPN"]
    
    mytokens = [ word if word.pos_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    
    #grab only my key words
    my_sub_tokens =[]
    for word in set(mytokens):
        if word in key_words:
            my_sub_tokens.append(word)

    # return preprocessed list of tokens
    return my_sub_tokens

#### CV

In [ ]:
cv = CountVectorizer(tokenizer=spacy_tokenizer_reverse,  stop_words='english', ngram_range=(1,1))

In [ ]:
dtm = cv.fit_transform(nlp_rope['info'].sample(n=1000, random_state=1))

#### LDA Model

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=6,               # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=32,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

print(lda_model)  # Model attributes

In [ ]:
# This can take awhile, we're dealing with a large amount of documents!

lda_output = lda_model.fit_transform(dtm)


#### Diagnose model performance with perplexity and log-likelihood
A model with higher log-likelihood and lower perplexity (exp(-1. * log-likelihood per word)) is considered to be good. Let’s check for our model.



In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

#### Top words for all groups

In [ ]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')